# Working with configuration

 ---
## Table of contents

- [Built-in configurations](#built_in_configurations)
- [User defined configurations](#user_defined_configurations)

In [ ]:
# lint with black
%load_ext lab_black

In [88]:
from bids import BIDSLayout
from bids.tests import get_test_data_path
import os

<a id='built_in_configurations'></a>
## Built-in configurations

If we try to load a derivatives data by itself.

In [73]:
data_path = os.path.join(get_test_data_path(), "synthetic", "derivatives", "fmriprep")

Without `validate=False` no subject is detected.

In [33]:
layout = BIDSLayout(data_path)
print(layout)

BIDS Layout: ...synthetic/derivatives/fmriprep | Subjects: 0 | Sessions: 0 | Runs: 0


Note that this is equivalent to using the default pybids configuration and that none of the typical entities associated with derivatives (like `desc` for description) would be availaible for querying this layout or for parsing files.

In [78]:
layout = BIDSLayout(data_path, config=['bids'])
print(layout)
print(layout.get_entities().keys())
"desc" in layout.get_entities()

BIDS Layout: ...synthetic/derivatives/fmriprep | Subjects: 0 | Sessions: 0 | Runs: 0
dict_keys(['subject', 'session', 'task', 'acquisition', 'ceagent', 'reconstruction', 'direction', 'run', 'proc', 'modality', 'echo', 'flip', 'inv', 'mt', 'part', 'recording', 'space', 'suffix', 'scans', 'fmap', 'datatype', 'extension'])


False

In [80]:
filename="sub-01_ses-01_task-nback_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"
# no `desc` entity reported
layout.parse_file_entities(filename)

{'session': '01',
 'task': 'nback',
 'run': 1,
 'space': 'MNI152NLin2009cAsym',
 'suffix': 'bold',
 'extension': '.nii.gz'}

But we can add the `derivatives` configuration to change this behavior.

In [74]:
layout = BIDSLayout(data_path, config=['bids', 'derivatives'])
print(layout)
print(layout.get_entities().keys())
"desc" in layout.get_entities()

BIDS Layout: ...synthetic/derivatives/fmriprep | Subjects: 5 | Sessions: 10 | Runs: 10
dict_keys(['subject', 'session', 'task', 'acquisition', 'ceagent', 'reconstruction', 'direction', 'run', 'proc', 'modality', 'echo', 'flip', 'inv', 'mt', 'part', 'recording', 'space', 'suffix', 'scans', 'fmap', 'datatype', 'extension', 'atlas', 'roi', 'label', 'desc', 'from', 'to', 'mode', 'hemi', 'res', 'den', 'model', 'subset', 'TaskName', 'RepetitionTime'])


True

And we can now query this layout for derivatives entities.

In [75]:
layout.get(extension="nii.gz", subject="01", suffix="bold", desc="preproc", space="MNI152NLin2009cAsym", return_type="filename")

['/home/remi/github/pybids/bids/tests/data/synthetic/derivatives/fmriprep/sub-01/ses-01/func/sub-01_ses-01_task-nback_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz',
 '/home/remi/github/pybids/bids/tests/data/synthetic/derivatives/fmriprep/sub-01/ses-01/func/sub-01_ses-01_task-nback_run-02_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz',
 '/home/remi/github/pybids/bids/tests/data/synthetic/derivatives/fmriprep/sub-01/ses-01/func/sub-01_ses-01_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz',
 '/home/remi/github/pybids/bids/tests/data/synthetic/derivatives/fmriprep/sub-01/ses-02/func/sub-01_ses-02_task-nback_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz',
 '/home/remi/github/pybids/bids/tests/data/synthetic/derivatives/fmriprep/sub-01/ses-02/func/sub-01_ses-02_task-nback_run-02_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz',
 '/home/remi/github/pybids/bids/tests/data/synthetic/derivatives/fmriprep/sub-01/ses-02/func/sub-01_ses-02_task-rest_s

And parse files for derivatives only entities.

In [76]:
filename="sub-01_ses-01_task-nback_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"
layout.parse_file_entities(filename)

{'session': '01',
 'task': 'nback',
 'run': 1,
 'space': 'MNI152NLin2009cAsym',
 'suffix': 'bold',
 'extension': '.nii.gz',
 'desc': 'preproc'}

<a id='user_defined_configurations'></a>
## User defined configurations

It is also possible to pass the path to a specific bids defined in a JSON file to override the defaults.

In [48]:
config_path = os.path.join(get_test_data_path(), "..", 'bids_specs_with_oligarchy.json')
layout = BIDSLayout(data_path, config=[config_path, "derivatives"])
print(layout.get_entities().keys())
"oligarchy" in layout.get_entities()

dict_keys(['subject', 'oligarchy', 'task', 'acquisition', 'ceagent', 'reconstruction', 'direction', 'run', 'proc', 'modality', 'echo', 'recording', 'space', 'suffix', 'scans', 'fmap', 'datatype', 'extension', 'atlas', 'roi', 'label', 'desc', 'from', 'to', 'mode', 'hemi', 'res', 'den', 'model', 'subset', 'TaskName', 'RepetitionTime'])


True

## Parsing files with a specific configuration

In [89]:
from bids.layout.models import Config

In [93]:
data_path = os.path.join(get_test_data_path(), "synthetic")
filename = '/sub-03_ses-07_run-4_desc-preproc_bold.nii.gz'

In [94]:
layout = BIDSLayout(data_path)
config = Config.load('bids')
layout.parse_file_entities(filename, config=[config])

{'subject': '03',
 'session': '07',
 'run': 4,
 'suffix': 'bold',
 'extension': '.nii.gz'}

In [95]:
layout = BIDSLayout(data_path)
layout.parse_file_entities(filename, config='bids')

{'subject': '03',
 'session': '07',
 'run': 4,
 'suffix': 'bold',
 'extension': '.nii.gz'}

# TODO
make parsing with config work

In [98]:
def test_parse_file_entities_from_layout(layout_synthetic):
    layout = layout_synthetic
    filename = '/sub-03_ses-07_run-4_desc-bleargh_sekret.nii.gz'

    # Test with entities taken from bids config
    target = {'subject': '03', 'session': '07', 'run': 4, 'suffix': 'sekret',
              'extension': '.nii.gz'}
    assert target == layout.parse_file_entities(filename, config='bids')
    config = Config.load('bids')
    assert target == layout.parse_file_entities(filename, config=[config])
    assert target == layout.parse_file_entities(filename, scope='raw')

    # Test with default scope--i.e., everything
    target = {'subject': '03', 'session': '07', 'run': 4, 'suffix': 'sekret',
              'desc': 'bleargh', 'extension': '.nii.gz'}
    assert target == layout.parse_file_entities(filename)
    # Test with only the fmriprep pipeline (which includes both configs)
    assert target == layout.parse_file_entities(filename, scope='fmriprep')
    assert target == layout.parse_file_entities(filename, scope='derivatives')

In [96]:
layout = BIDSLayout(data_path, derivatives=True)
layout.parse_file_entities(filename, config='derivatives', scope='fmriprep')

{'desc': 'preproc'}